In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))



In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] = 'gray'
import plotly.express as px
import seaborn as sns

from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor as rfr, ExtraTreesRegressor as etr
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor as lgb

import warnings
warnings.simplefilter('ignore')



In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
train.time = pd.to_datetime(train.time)
train['time_id'] = ( ( (train.time.dt.dayofyear-1)*24*60 + train.time.dt.hour*60 + train.time.dt.minute ) /20 ).astype(int)
train = train.set_index('row_id', drop=True)
train.head(3)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id')
test.time = pd.to_datetime(test.time)

submission = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:


train['dir_xy'] = train.direction.map({'EB':'[1,0]', 'NB':'[0,1]', 'SB':'[0,-1]', 'WB':'[-1,0]', 'NE':'[1,1]', 'SW':'[-1,-1]', 'NW':'[-1,1]', 'SE':'[1,-1]'})



In [ ]:


loc_dir = train.groupby(['x','y']).dir_xy.unique().reset_index()
loc_dir['num_dir'] = loc_dir.dir_xy.apply(lambda x: len(x))
print(loc_dir)
print(f'Number of roadways: {loc_dir.num_dir.sum()}')



In [ ]:


plt.figure(figsize=(10,7), facecolor='gray')

for idx, row in loc_dir.iterrows():
    origin = np.repeat(np.array([[row.x],[row.y]]),row.num_dir, axis=-1)
    V = np.array([eval(s) for s in row.dir_xy])
    
    plt.quiver(*origin, V[:,0], V[:,1], scale=15, color='white')
plt.xlim(-0.5,2.5)
plt.ylim(-0.5,3.5)
plt.xticks([])
plt.yticks([])
plt.show()



In [ ]:
train.head()

In [ ]:
train = train.drop('dir_xy',axis=1)

In [ ]:
print(train.groupby(['x', 'y', 'direction']).time.count())
print(f'Total time instants within the observation period: {int(train.time_id.max() - train.time_id.min()+1)}')

### 13059 observations for each roadway 

In [ ]:
time = pd.DataFrame(pd.date_range('1991-04-01 00:00:00', '1991-09-30 11:40:00', freq='20Min'), columns=['time'])

dfs = []
for group in train.groupby(['x', 'y', 'direction']):
    dfs.append(group[1].merge(time, on='time', how= 'right'))

train_new = pd.concat(dfs, axis=0).sort_values(['time', 'x', 'y', 'direction']).reset_index()
print(f'Missing row:\n{train_new.isna().sum()}')

### 5265 missing instants

In [ ]:
train_new

Visualization

In [ ]:


# Histogram
plt.figure(figsize=(10,5))
train.congestion.hist(bins=100, color='yellow')
plt.xlabel('Congestion', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.show()



## Morning vs Afternoon

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
train[train.time.dt.hour<12].congestion.hist(bins=100, color='orange', label='Morning')
train[train.time.dt.hour>=12].congestion.hist(bins=100, color='yellow', label='Afternoon')
plt.xlabel('Congestion', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.legend()

plt.subplot(1,2,2)
train[train.time.dt.hour>=12].congestion.hist(bins=100, color='yellow', label='Afternoon')
train[train.time.dt.hour<12].congestion.hist(bins=100, color='orange', label='Morning')
plt.xlabel('Congestion', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.legend()

plt.show()

In [ ]:
#If we add 5 congestion units to the moning data, its distribution fits quite well with that of the afternoon
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
(train[train.time.dt.hour<12].congestion+5).hist(bins=100, color='orange', label='Morning+5')
train[train.time.dt.hour>=12].congestion.hist(bins=100, color='yellow', label='Afternoon')
plt.xlabel('Congestion', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.legend()

plt.subplot(1,2,2)
train[train.time.dt.hour>=12].congestion.hist(bins=100, color='yellow', label='Afternoon')
(train[train.time.dt.hour<12].congestion+5).hist(bins=100, color='orange', label='Morning+5')
plt.xlabel('Congestion', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.legend()

plt.show()

In [ ]:
#Histograms for each roadway
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,12))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:
                plt.subplot(4,2,idx+1)
                df.congestion.hist(bins=100, color='yellow')
                plt.title(f'x={x}, y={y}, direction={direction}', fontsize=16)
                plt.xlabel('Congestion', fontsize=16)
                plt.ylabel('Count', fontsize=16)
                plt.tight_layout()
            
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

### Correlation between 12 locations

In [ ]:


dfs = []
columns = []
for x in range(3):
    for y in range(4):
        df = train[(train.x == x) & (train.y==y)]
        dfs.append(df.groupby('time').congestion.mean().tolist())
        columns.append(f'x{x}y{y}')      
location_congestions = pd.DataFrame(np.array(dfs).T, columns=columns)

plt.figure(figsize=(12,10))
sns.heatmap(location_congestions.corr(), annot=True)
plt.title('Correlation between the congestion of 12 locations', fontsize=16)
plt.xticks(rotation=90, fontsize=16)
plt.yticks(rotation=0, fontsize=16)
plt.show()



### strong correlation is crossover more than or eqaul to 0.7

## Correlation between 12 locations at two consecutive instants

In [ ]:

### hour and min 
plt.figure(figsize=(18,8))

# At 12h
dfs = []
columns = []
for x in range(3):
    for y in range(4):
        df = train[(train.x == x) & (train.y==y) & (train.time.dt.hour==8) & (train.time.dt.minute==0)]
        dfs.append(df.groupby('time').congestion.mean().tolist())
        columns.append(f'x{x}y{y}')      
location_congestions = pd.DataFrame(np.array(dfs).T, columns=columns)

plt.subplot(1,2,1)
sns.heatmap(location_congestions.corr(), annot=True)
plt.title('Correlation between the locations at 8h', fontsize=16)
plt.xticks(rotation=90, fontsize=16)
plt.yticks(rotation=90, fontsize=16)
plt.tight_layout()

# At 12h20'
dfs = []
columns = []
for x in range(3):
    for y in range(4):
        df = train[(train.x == x) & (train.y==y) & (train.time.dt.hour==8) & (train.time.dt.minute==20)]
        dfs.append(df.groupby('time').congestion.mean().tolist())
        columns.append(f'x{x}y{y}')      
location_congestions = pd.DataFrame(np.array(dfs).T, columns=columns)

plt.subplot(1,2,2)
sns.heatmap(location_congestions.corr(), annot=True)
plt.title('Correlation between the locations at 8h20', fontsize=16)
plt.xticks(rotation=90, fontsize=16)
plt.yticks(rotation=0, fontsize=16)
plt.tight_layout()

plt.show()

## Daily correlation between the locations (x,y)=(1,2) and (1,3)

In [ ]:
train['daytime_id'] = ( (train.time.dt.hour*60+train.time.dt.minute) /20 ).astype(int)

In [ ]:
train.head()

In [ ]:
corrs = []
for daytime_id in train.daytime_id.unique():
    loc_12 = train[(train.x == 1) & (train.y==2) & (train.daytime_id==daytime_id)].groupby('time').congestion.mean().tolist()
    loc_13 = train[(train.x == 1) & (train.y==3) & (train.daytime_id==daytime_id)].groupby('time').congestion.mean().tolist()
    corr = pd.DataFrame(np.array([loc_12, loc_13]).T).corr()[0][1]
    corrs.append(corr)
    
plt.figure(figsize=(10,7))
plt.plot(train.daytime_id.unique()/3.0, corrs, 'white', linewidth=3)
plt.xlabel('Hour', fontsize=16)
plt.ylabel('Correlation coefficient', fontsize=16)
plt.title('Correlation between locations (x,y)=(1,2) and (1,3)', fontsize=16)
plt.text(8.5, 0.85, '8am', color='blue', fontsize=16)
plt.text(18, 0.65, '5pm', color='blue', fontsize=16)
plt.show()

In [ ]:
#Correlation between different directions at each location
plt.figure(figsize=(12,36))
for x in range(3):
    for y in range(4):
        dfs = []
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                dfs.append(df.congestion.tolist())
                
        roadway_congestions = pd.DataFrame(np.array(dfs).T, columns=train[(train.x == x) & (train.y==y)].direction.unique())
        
        plt.subplot(6,2,4*x+y+1)
        sns.heatmap(roadway_congestions.corr(), annot=True)
        plt.title(f'Correlation between {len(dfs)} directions at x={x}, y={y}', fontsize=16)
        plt.tight_layout()
plt.show()

# Average congestion per month

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(train.time.dt.month.unique(), train.groupby(train.time.dt.month).congestion.mean(), color='y')
plt.title(f'Mean congestion per month', fontsize=16)
plt.xlabel('Month', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.ylim(45,50)
plt.show()

In [ ]:
#Average congestion per month for each roadway
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.bar(df.time.dt.month.unique(), df.groupby(df.time.dt.month).congestion.mean(), color='y')
                plt.title(f'Mean congestion per month at x={x}, y={y}, direction={direction}')
                plt.xlabel('Month', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)
                plt.ylim(df.groupby(df.time.dt.month).congestion.mean().round().astype(int).min()-1, df.groupby(df.time.dt.month).congestion.mean().round().astype(int).max()+1)
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

# Average montly congestion

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(train.time.dt.day.unique(), train.groupby(train.time.dt.day).congestion.mean(), color='y')
plt.title(f'Mean monthly congestion', fontsize=16)
plt.xlabel('Day of month', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.ylim(45,50)
plt.show()

# Average montly congestion for each roadway

In [ ]:
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.bar(df.time.dt.day.unique(), df.groupby(df.time.dt.day).congestion.mean(), color='y')
                plt.title(f'Mean monthly congestion at x={x}, y={y}, direction={direction}')
                plt.xlabel('Day of month', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)
                plt.ylim(df.groupby(df.time.dt.day).congestion.mean().round().astype(int).min()-1, df.groupby(df.time.dt.day).congestion.mean().round().astype(int).max()+1)
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

# Average congestion per week

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(train.time.dt.week.unique(), train.groupby(train.time.dt.week).congestion.mean(), color='y')
plt.title(f'Mean congestion per week', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.ylim(45,50)
plt.show()

# Average congestion per week for each roadway

In [ ]:
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.bar(df.time.dt.week.unique(), df.groupby(df.time.dt.week).congestion.mean(), color='y')
                plt.title(f'Mean congestion per week at x={x}, y={y}, direction={direction}')
                plt.xlabel('Week', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)
                plt.ylim(df.groupby(df.time.dt.week).congestion.mean().round().astype(int).min()-1, df.groupby(df.time.dt.week).congestion.mean().round().astype(int).max()+1)
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

In [ ]:
#Average weekly congestion

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(train.time.dt.weekday.unique(), train.groupby(train.time.dt.weekday).congestion.mean(), color='y')
plt.title(f'Mean weely congestion', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.xticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], fontsize=16, rotation=90)
plt.ylim(40,50)
plt.show()

# Average weekly congestion for each roadway

In [ ]:
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.bar(df.time.dt.weekday.unique(), df.groupby(df.time.dt.weekday).congestion.mean(), color='y')
                plt.title(f'Mean weekly congestion at x={x}, y={y}, direction={direction}')
                plt.ylabel('Congestion', fontsize=16)
                plt.ylim(df.groupby(df.time.dt.weekday).congestion.mean().round().astype(int).min()-1, df.groupby(df.time.dt.weekday).congestion.mean().round().astype(int).max()+1)
                plt.xticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], fontsize=16, rotation=90)
                plt.tight_layout()
        plt.show()
                
        print('\n\n')
        break # remove break to plot for all the 65 roadways

In [ ]:
#Average congestion per day
plt.figure(figsize=(10,7))
plt.bar(train.time.dt.dayofyear.unique(), train.groupby(train.time.dt.dayofyear).congestion.mean(), color=['r' if (int(day)%7)==6 else 'y' for day in train.time.dt.dayofyear.unique()])
plt.title(f'Mean congestion per day', fontsize=16)
plt.xlabel('Day of year (Sunday is marked by red color)', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.ylim(40,50)
plt.show()

In [ ]:
#Average congestion per day for each roadway
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.bar(df.time.dt.dayofyear.unique(), df.groupby(df.time.dt.dayofyear).congestion.mean(), color=['r' if (int(day)%7)==6 else 'y' for day in train.time.dt.dayofyear.unique()])
                plt.title(f'Mean congestion per day at x={x}, y={y}, direction={direction}')
                plt.xlabel('Day of year (Sunday is marked by red color)', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)                
                plt.ylim(df.groupby(df.time.dt.dayofyear).congestion.mean().round().astype(int).min()-1, df.groupby(df.time.dt.dayofyear).congestion.mean().round().astype(int).max()+1)
                plt.tight_layout()
        plt.show()
                
        print('\n\n')
        break # remove break to plot for all the 65 roadways

# Average daily congestion

In [ ]:

plt.figure(figsize=(10,7))
plt.plot(train.groupby(train.daytime_id).daytime_id.unique()/3.0, train.groupby(train.daytime_id).congestion.mean(), 'y', linewidth=5)
plt.title(f'Mean daily congestion', fontsize=16)
plt.xlabel('Hour', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.show()

# Average daily congestion for each roadway

In [ ]:

for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.plot(df.groupby(df.daytime_id).daytime_id.unique()/3.0, df.groupby(df.daytime_id).congestion.mean(), 'y', linewidth=3, label='Daily average')
                plt.plot(df[-36:].daytime_id/3.0, df[-36:].congestion, 'r', linewidth=3, label='The Monday 30 Sept')
                plt.title(f'Mean daily congestion at x={x}, y={y}, direction={direction}')
                plt.xlabel('Hour', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)
                plt.legend()
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

# Average Monday congestion

In [ ]:

df = train[train.time.dt.weekday==0]

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(df.groupby(df.daytime_id).daytime_id.unique()/3.0, df.groupby(df.daytime_id).congestion.mean(), 'y', linewidth=3, label='Mean of Mondays')

plt.plot(train.groupby(train.daytime_id).daytime_id.unique()/3.0, train.groupby(train.daytime_id).congestion.median(), 'orange', linewidth=3, label='Mean of all days')
plt.title(f'Mean of Monday congestion', fontsize=16)
plt.xlabel('Hour', fontsize=16)
plt.ylabel('Congestion', fontsize=16)
plt.legend()
plt.show()

# Average Monday congestion for each roadway

In [ ]:
for x in range(3):
    for y in range(4):
        plt.figure(figsize=(12,24))
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each location and direction
            df = train[(train.time.dt.weekday==0) & (train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                plt.subplot(4,2,idx+1)
                plt.plot(df.groupby(df.daytime_id).daytime_id.unique()/3.0, df.groupby(df.daytime_id).congestion.median(), 'y', linewidth=3, label='Mean Monday')
                plt.plot(df[-36:].daytime_id/3.0, df[-36:].congestion, 'r', linewidth=3, label='The Monday 30 Sept')
                plt.title(f'x={x}, y={y}, direction={direction}')
                plt.xlabel('Hour', fontsize=16)
                plt.ylabel('Congestion', fontsize=16)
                plt.legend()
        plt.show()
        
        print('\n\n')
        break # remove break to plot for all the 65 roadways

# Correlation between the days

In [ ]:
train_new.congestion = train_new.congestion.fillna(train_new.congestion.mean())

In [ ]:
congestion = pd.DataFrame(np.array(train_new.congestion).reshape(365,2340).T)
congestion_morning = congestion[[col for col in congestion.columns if col%2==0]]
congestion_afternoon = congestion[[col for col in congestion.columns if col%2==1]]
congestion_monday_afternoon = congestion[[col for col in congestion.columns if col%14==1]].round().astype(int)
congestion_monday_afternoon_smoothed = congestion_monday_afternoon.rolling(10, min_periods=1).mean()

In [ ]:
plt.figure(figsize=(18,7))

plt.subplot(1,2,1)
sns.heatmap(congestion_monday_afternoon[congestion_monday_afternoon.columns[-10:]].corr(), annot=True)
plt.title('Correlation between the last 10 Monday afternoons', fontsize=16)

plt.subplot(1,2,2)
sns.heatmap(congestion_monday_afternoon_smoothed[congestion_monday_afternoon_smoothed.columns[-10:]].corr(), annot=True)
plt.title('Smoothed', fontsize=16)

plt.show()

In [ ]:
#Score between the congestion of the Monday afternoons

In [ ]:
df = congestion_monday_afternoon
scores=[]
for idx in df.columns:
    score = 0
    for time_id in range(36):
        score +=mean_absolute_error(df[idx][(time_id*65):((time_id+1)*65)].astype(int), df[df.columns[-1]][(time_id*65):((time_id+1)*65)].astype(int))
    scores.append(score/36.0)

plt.figure(figsize=(10,7))
plt.bar(range(len(scores)),scores, color='y')
plt.xlabel('The Monday afternoons', fontsize=16)
plt.ylabel('MAE vs the Monday afternoon 23 Sept', fontsize=16)
plt.show()

## Score between the congestion of the Monday afternoons w.r.t. the median congestion

In [ ]:
df = congestion_afternoon
df_monday = congestion_monday_afternoon
scores=[]
for idx in df_monday.columns:
    score =mean_absolute_error(df_monday[idx].astype(int), np.median(df, axis=1).round().astype(int))
    scores.append(score)
plt.figure(figsize=(10,7))
plt.bar(range(len(scores)),scores, color='y')
plt.xlabel('The Monday afternoons', fontsize=16)
plt.ylabel('MAE vs the median of all the afternoons', fontsize=16)
plt.show()

## Score between the congestion of the afternoons

In [ ]:
df = congestion_afternoon
scores=[]
for idx in df.columns:
    score =mean_absolute_error(df[idx].astype(int), df[df.columns[-1]].astype(int))
    scores.append(score)
plt.figure(figsize=(10,7))
plt.bar(range(len(scores)),scores, color='y')
plt.xlabel('The afternoons', fontsize=16)
plt.ylabel('MAE vs the afternoon 29 Sept', fontsize=16)
plt.show()

In [ ]:
#Score between the congestion of the afternoons w.r.t. the median congestion

In [ ]:
df = congestion_afternoon
scores=[]
for idx in df.columns:
    score =mean_absolute_error(df[idx].astype(int), np.median(df, axis=1).round().astype(int))
    scores.append(score)
plt.figure(figsize=(10,7))
plt.bar(range(len(scores)),scores, color='y')
plt.xlabel('The afternoons', fontsize=16)
plt.ylabel('MAE vs the median of all the afternoons', fontsize=16)
plt.show()

In [ ]:
#The ouliers

In [ ]:
df = congestion_afternoon
outliers=[]
outlier_days=[]
for idx in df.columns:
    if (mean_absolute_error(df[idx].astype(int), np.median(df, axis=1).round().astype(int)) > 7):
        day_idx = idx //2 + 91
        
        outliers.append(idx)
        outlier_days.append(day_idx)
print(f'% outliers: {len(outliers)/len(df.columns)*100}')

In [ ]:
df = congestion_afternoon[[idx for idx in congestion_afternoon.columns if idx not in outliers]]
scores=[]
for idx in df.columns:
    score =mean_absolute_error(df[idx].astype(int), np.median(df, axis=1).round().astype(int))
    scores.append(score)
plt.figure(figsize=(10,7))
plt.bar(range(len(scores)),scores, color='y')
plt.xlabel('The afternoons', fontsize=16)
plt.ylabel('MAE vs the median of all the afternoons', fontsize=16)
plt.show()

In [ ]:
#Simple baselines without Machine Learning

# Daily average

In [ ]:
test['congestion'] = -1

for x in range(3):
    for y in range(4):
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each roadway
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                test.congestion[(test.x == x) & (test.y==y) & (test.direction==direction)] = df.groupby(df.daytime_id).congestion.median().tolist()[-36:]

In [ ]:
#Daily average, exclude weekend

In [ ]:
test['congestion'] = -1

for x in range(3):
    for y in range(4):
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each roadway
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                test.congestion[(test.x == x) & (test.y==y) & (test.direction==direction)] = df[df.time.dt.weekday<=4].groupby(df.daytime_id).congestion.median().tolist()[-36:]

In [ ]:
submission.congestion = test.congestion.round().astype(int).tolist()
submission.congestion[submission.congestion<0] = 0
submission.congestion[submission.congestion>100] = 100
submission.to_csv('dailyMedianExcludeWeekend.csv', index=False)
submission.head()

# Daily average, remove outliers

In [ ]:
test['congestion'] = -1

for x in range(3):
    for y in range(4):
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each roadway
            df = train[(train.x == x) & (train.y==y) & (train.direction==direction) & (~train.time.dt.dayofyear.isin(outlier_days))]
            
            if df.shape[0]>0:                
                test.congestion[(test.x == x) & (test.y==y) & (test.direction==direction)] = df.groupby(df.daytime_id).congestion.median().tolist()[-36:]

In [ ]:
submission.congestion = test.congestion.round().astype(int).tolist()
submission.congestion[submission.congestion<0] = 0
submission.congestion[submission.congestion>100] = 100
submission.to_csv('dailyMedianRemoveOutliers.csv', index=False)
submission.head()

In [ ]:
submission.congestion = test.congestion.round().astype(int).tolist()
submission.congestion[submission.congestion<0] = 0
submission.congestion[submission.congestion>100] = 100
submission.to_csv('dailyMedian.csv', index=False)
submission.head()

# Average of all Monday

In [ ]:
test['congestion'] = -1

for x in range(3):
    for y in range(4):
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each roadway
            df = train[(train.time.dt.weekday==0) & (train.x == x) & (train.y==y) & (train.direction==direction)]
            
            if df.shape[0]>0:                
                test.congestion[(test.x == x) & (test.y==y) & (test.direction==direction)] = df.groupby(df.daytime_id).congestion.mean().tolist()[-36:]

In [ ]:


submission.congestion = test.congestion.round().astype(int).tolist()
submission.congestion[submission.congestion<0] = 0
submission.congestion[submission.congestion>100] = 100
submission.to_csv('MondayMean.csv', index=False)
submission.head()



# Daily Linear Regression

In [ ]:
test['congestion'] = -1

for x in range(3):
    for y in range(4):
        for idx, direction in enumerate(train.direction.unique()):
            # extract data for each roadway
            df = train_new[(train_new.x == x) & (train_new.y==y) & (train_new.direction==direction)]
            df_new = pd.DataFrame()
            if df.shape[0]>0:                
                for idx, group in df.groupby(df.time.dt.dayofyear):
                    if len(group.congestion) == 72: # consider only days without missing data
                        df_new[f'day_{idx}'] = group.congestion.tolist()
            
                X_train = df_new[df_new.columns[:-1]][-36:]
                y_train = df_new[df_new.columns[-1]][-36:]

                X_test = df_new[df_new.columns[1:]][-36:]

                model = lr()
                model.fit(X_train, y_train)

                test.congestion[(test.x == x) & (test.y==y) & (test.direction==direction)] = model.predict(X_test).tolist()

In [ ]:
submission.congestion = test.congestion.round().astype(int).tolist()
submission.congestion[submission.congestion<0] = 0
submission.congestion[submission.congestion>100] = 100
submission.to_csv('DailyLinearRegression.csv', index=False)
submission.head()